<b>Google Colab</b> <a href="https://colab.research.google.com/github/kirillzyusko/deeplearning/blob/master/7/lab.ipynb">link</a>

Authorize google + kaggle

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


Be sure, that we authorized and have an access to kaggle

In [3]:
%ls /content/.kaggle/

kaggle.json*


# **Part 1: Download dataset, extract**

In [13]:
!wget "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

--2020-04-03 16:58:45--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  35.0MB/s    in 2.3s    

2020-04-03 16:58:47 (35.0 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [0]:
!mkdir data
!tar xvzf aclImdb_v1.tar.gz -C data

In [0]:
import os

imdb_dir = 'data/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [44]:
labels.shape

(25000,)

In [45]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 20 words
training_samples = 12500  # We will be training on 25000 samples
validation_samples = 12500  # We will be validating on 25000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [38]:
x_train[1]

array([1329,    2,  122,   46, 3921,    2,    7,    7, 1334, 1080,    9,
          7,    7, 6400,  149, 1533,  165,   37,    9,   13,  811,   20,
        371,    9,   13,    7,    7,  811,   20, 4455,  341, 5811,    1,
       1053,   66,    7,    7, 1853,    4, 1519,   52, 1520, 1853,    2,
       7312,    7,    7,  162, 1776,  292,    8,   46,  136,   18,  666,
         22,  188,  839,    7,    7,    1,  153,   22,  188,  388,   48,
        503,  396,    2,   22,  460,    7,    7,   51,    1,  372,  211,
         24,   36,   24,  643,    7,    7,  137,    2,  849,   11,   17,
       1424,  105,   37,   11,   33,   23,    3, 6911,    2,    7,    7,
          3], dtype=int32)

In [46]:
x_val[0]

array([  20,  285,   12, 5445,    7,    7,    1,  236,   31, 2272,  248,
       2074,    6,   28,    4,    1,  830,  865,  113, 2628,  204,  123,
        107,  368, 1998, 1338,    7,    7,   58,  314, 2022, 3309,   50,
         71,   46, 7630,    5,    3,   50, 1133,   19,    1, 8116, 4026,
          1,    8,   12,  254, 2235,    1,  110,    4,    3,  144,  411,
       2962,    1,    2, 6502,    4,   32,  573, 4353, 1742,    7,    7,
          2,    8,  396,   35,    9,   16, 1019, 8294,   60,  775,   43,
          4,    1,  311,   12, 3382,   69,  325,   14,    1, 4790,   36,
       5676,    5, 5676,   10, 1291,  841, 6111,  200,   27,   15,   58,
       1352], dtype=int32)

Download dataset:

In [0]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/kaggle/imdb

 70% 18.0M/25.7M [00:00<00:00, 28.6MB/s]
100% 25.7M/25.7M [00:00<00:00, 40.5MB/s]


Extract .zip

In [0]:
!unzip kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip -d data

Archive:  kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip
  inflating: data/IMDB Dataset.csv   


Files:

In [0]:
%ls data

'IMDB Dataset.csv'


Read data using pandas:

In [0]:
import pandas as pd

imdb_ds = pd.read_csv('data/IMDB Dataset.csv')
print(imdb_ds.shape)

imdb_ds.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
!wget https://raw.githubusercontent.com/cdipaolo/sentiment/master/datasets/imdb.vocab

--2020-03-18 13:31:22--  https://raw.githubusercontent.com/cdipaolo/sentiment/master/datasets/imdb.vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 845980 (826K) [text/plain]
Saving to: ‘imdb.vocab’

imdb.vocab          100%[===================>] 826.15K  --.-KB/s    in 0.09s   

2020-03-18 13:31:22 (8.77 MB/s) - ‘imdb.vocab’ saved [845980/845980]



In [0]:
with open("imdb.vocab", "r") as file:
  data = file.read()
  print(data)

.........................................

In [1]:
from keras.datasets import imdb

VOCAB_SIZE = 10000

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

Using TensorFlow backend.


In [23]:
print(train_data.shape)
print(test_data.shape)

(25000,)
(25000,)


In [37]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


# **Part 2:**

In [51]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional

model = Sequential()
model.add(Embedding(VOCAB_SIZE, 8, input_length=maxlen))
model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1)
score, acc = model.evaluate(x_val, y_val)

print('Test accuracy:', acc)

Epoch 1/10
391/391 [==============================] - 66s 168ms/step - loss: 0.6933 - accuracy: 0.5006
Epoch 2/10
391/391 [==============================] - 63s 160ms/step - loss: 0.6826 - accuracy: 0.5716
Epoch 3/10
391/391 [==============================] - 62s 159ms/step - loss: 0.5788 - accuracy: 0.6989
Epoch 4/10
391/391 [==============================] - 62s 157ms/step - loss: 0.4378 - accuracy: 0.8014
Epoch 5/10
391/391 [==============================] - 62s 159ms/step - loss: 0.3267 - accuracy: 0.8586
Epoch 6/10
391/391 [==============================] - 65s 166ms/step - loss: 0.2516 - accuracy: 0.8964
Epoch 7/10
391/391 [==============================] - 62s 160ms/step - loss: 0.1943 - accuracy: 0.9229
Epoch 8/10
391/391 [==============================] - 62s 158ms/step - loss: 0.1577 - accuracy: 0.9402
Epoch 9/10
391/391 [==============================] - 62s 159ms/step - loss: 0.1294 - accuracy: 0.9503
Epoch 10/10
391/391 [==============================] - 8s 21ms/step - los

# **Part 3: Using GloVe**

In [19]:
!wget "http://nlp.stanford.edu/data/glove.6B.zip"

--2020-04-03 17:06:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-03 17:06:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-03 17:06:42--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [20]:
!mkdir glove
!unzip glove.6B.zip -d glove

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [21]:
glove_dir = 'glove'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [48]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1)
score, acc = model.evaluate(x_val, y_val)

print('Test accuracy:', acc)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 20, 100)           1000000   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 64)                34048     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 1,034,113
Trainable params: 1,034,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
391/391 [==============================] - 73s 187ms/step - loss: 0.6969 - accuracy: 0.5062
Epoch 2/10
391/391 [==============================] - 73s 186ms/step - loss: 0.6918 - accuracy: 0.5226
Epoch 3/10
391/391 [==============================] - 72s 185ms/step - loss: 0.6901 - accuracy: 0.5279
Epoch 4/10
391/391 [==============================] - 72s 185ms/s

# **Part 4: New NN architecture**

In [53]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, embedding_dim, input_length=max_len))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1)
score, acc = model.evaluate(x_val, y_val)

print('Test accuracy:', acc)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 20, 100)           1000000   
_________________________________________________________________
lstm_13 (LSTM)               (None, 32)                17024     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 1,018,113
Trainable params: 1,018,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
237/391 [=================>............] - ETA: 15s - loss: 0.6946 - accuracy: 0.5001

KeyboardInterrupt: ignored

# **Part 5: DeepMoji**

In [0]:
!pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
!git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
!pip3 install -e .

In [0]:
!python3 scripts/download_weights.py

In [80]:
%cd ../../../../../

/


In [88]:
%ls

data/      emoji_overview.png  examples/  model/     scripts/  tests/
deepmoji/  emoji_unicode.csv   LICENSE    README.md  setup.py


In [90]:
!python scripts/download_weights.py

About to download the pretrained weights file from https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
Traceback (most recent call last):
  File "scripts/download_weights.py", line 52, in <module>
    if already_exists or prompt():
  File "scripts/download_weights.py", line 30, in prompt
    choice = raw_input().lower()
NameError: name 'raw_input' is not defined


In [0]:
"""Trains the DeepMoji architecture on the IMDB sentiment classification task.
   This is a simple example of using the architecture without the pretrained model.
   The architecture is designed for transfer learning - it should normally
   be used with the pretrained model for optimal performance.
"""
from __future__ import print_function
import example_helper
import numpy as np
from keras.preprocessing import sequence
from keras.datasets import imdb
from deepmoji.model_def import deepmoji_architecture

# Seed for reproducibility
np.random.seed(1337)

nb_tokens = 20000
maxlen = 80
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=nb_tokens)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = deepmoji_architecture(nb_classes=2, nb_tokens=nb_tokens, maxlen=maxlen)
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=15,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)